In [1]:
%pwd

'c:\\Code\\Project Prototypes\\GENAI MEDBOT\\evaluation'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Code\\Project Prototypes\\GENAI MEDBOT'

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4.1-nano-2025-04-14"

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='data/')

In [8]:
extracted_data

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250404205122', 'source': 'data\\Faida_Ya_Parachichi_Kwa_Mjamzito.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Faida Ya Parachichi Kwa Mjamzito.\n1) Kuimarisha Kinga Ya Mwili.\nVitamini C na E kwenye parachichi husaidia kuimarisha kinga ya mwili, ambayo ni muhimu kwa\nmjamzito ili kuzuia magonjwa.\n2) Kudhbiti Shinikizo La Damu.\nParachichi lina kiwango cha juu cha potasiamu, ambayo husaidia kudhibiti shinikizo la\ndamu.Shinikizo la damu lisilodhibitiwalinaweza kusababisha matatizo wakati wa ujauzito.\n3) Kuboresha Afya Ya Moyo.\nMafuta ya parachichi yanaweza kusaidia kupunguza kiwango cha cholesterol mbaya (Low Density\nLipopritein) na kuongeza cholesterol nzuri (High Density Lipoprotein), ambayo inasaidia kuboresha\nafya ya moyo.\n4) Kusaidia Kwenye Mmeng?enyo Wa Chakula.\nParachichi lina kiasi kikubwa cha nyuzinyuzi (fibers), ambazo husaidia k

In [9]:
import re
import unicodedata
from langchain.docstore.document import Document

def clean_text(text):
    # Normalize Unicode characters
    text = unicodedata.normalize("NFKC", text)
    
    # Replace encoding artifacts like stray question marks
    text = text.replace("?", " ")

    # Collapse multiple newlines to a single space
    text = re.sub(r'\n+', ' ', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [10]:
def preprocess_documents(documents):
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
    return documents

In [11]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
# Preprocess the raw PDFs
preprocessed_data = preprocess_documents(extracted_data)

In [13]:
preprocessed_data

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250404205122', 'source': 'data\\Faida_Ya_Parachichi_Kwa_Mjamzito.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Faida Ya Parachichi Kwa Mjamzito. 1) Kuimarisha Kinga Ya Mwili. Vitamini C na E kwenye parachichi husaidia kuimarisha kinga ya mwili, ambayo ni muhimu kwa mjamzito ili kuzuia magonjwa. 2) Kudhbiti Shinikizo La Damu. Parachichi lina kiwango cha juu cha potasiamu, ambayo husaidia kudhibiti shinikizo la damu.Shinikizo la damu lisilodhibitiwalinaweza kusababisha matatizo wakati wa ujauzito. 3) Kuboresha Afya Ya Moyo. Mafuta ya parachichi yanaweza kusaidia kupunguza kiwango cha cholesterol mbaya (Low Density Lipopritein) na kuongeza cholesterol nzuri (High Density Lipoprotein), ambayo inasaidia kuboresha afya ya moyo. 4) Kusaidia Kwenye Mmeng enyo Wa Chakula. Parachichi lina kiasi kikubwa cha nyuzinyuzi (fibers), ambazo husaidia katika kumeng

In [14]:
text_chunks=text_split(preprocessed_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 72


In [15]:
text_chunks # Displaying the first 1000 characters of the first chunk

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250404205122', 'source': 'data\\Faida_Ya_Parachichi_Kwa_Mjamzito.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Faida Ya Parachichi Kwa Mjamzito. 1) Kuimarisha Kinga Ya Mwili. Vitamini C na E kwenye parachichi husaidia kuimarisha kinga ya mwili, ambayo ni muhimu kwa mjamzito ili kuzuia magonjwa. 2) Kudhbiti Shinikizo La Damu. Parachichi lina kiwango cha juu cha potasiamu, ambayo husaidia kudhibiti shinikizo la damu.Shinikizo la damu lisilodhibitiwalinaweza kusababisha matatizo wakati wa ujauzito. 3) Kuboresha Afya Ya Moyo. Mafuta ya parachichi yanaweza kusaidia kupunguza kiwango cha cholesterol mbaya (Low Density Lipopritein) na kuongeza cholesterol nzuri (High Density Lipoprotein), ambayo inasaidia kuboresha afya ya moyo. 4) Kusaidia Kwenye Mmeng enyo Wa Chakula. Parachichi lina kiasi kikubwa cha nyuzinyuzi (fibers), ambazo husaidia katika kumeng

In [16]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

In [17]:
# #Download the Embeddings from Hugging Face
# def download_hugging_face_embeddings():
#     embeddings=HuggingFaceEmbeddings(model_name='sartifyllc/MultiLinguSwahili-bge-small-en-v1.5-nli-matryoshka')
#     return embeddings

In [18]:
# embeddings = download_hugging_face_embeddings()
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [19]:
#Load The Content into the Vector Store
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    text_chunks, embedding=embedding,
)

c:\Users\DELL\anaconda3\envs\llmapp\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [20]:
#Create a Knowledge Base
import pandas as pd

df = pd.DataFrame([d.page_content for d in text_chunks], columns=["text"])
df.head(5)

,text
0,Faida Ya Parachichi Kwa Mjamzito. 1) Kuimarish...
1,na afya ya moyo wa mama. 6) Foliki asidi (Vita...
2,bora kushauriana na daktari wako au mtaalamu w...
3,Faida Ya Tango Kwa Mama Mjamzito. Tango Kwa Mj...
4,na madoa. 4) Husaidia Katika Kudhibiti Shiniki...


In [21]:
# import pandas as pd

# # Add id to each document before creating the DataFrame
# for i, doc in enumerate(text_chunks):
#     doc.metadata["id"] = f"doc_{i}"  # or just str(i)

# # Then create the dataframe
# df1 = pd.DataFrame([{"id": d.metadata["id"], "text": d.page_content} for d in text_chunks])
# df1.head(10)

In [22]:
import giskard
print(giskard.__version__)

2.16.2


In [23]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

c:\Users\DELL\anaconda3\envs\llmapp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-24 14:29:48,925 pid:14944 MainThread giskard.llm.embeddings INFO     No embedding model set though giskard.llm.set_embedding_model. Defaulting to openai/text-embedding-3-small since OPENAI_API_KEY is set.


In [24]:
print(f"Knowledge base has {len(knowledge_base._documents)} documents")

Knowledge base has 72 documents


In [25]:
print(knowledge_base._documents[0])

In [26]:
doc = knowledge_base._documents[49]
print(doc.__dict__)

{'content': 'mpango wa mazoezi. 5) Kupata Usingizi Wa Kutosha. Usingizi wa kutosha ni muhimu kwa afya ya mama na mtoto aliye tumboni. Mama mjamzito anapaswa kupata usingizi wa kutosha na kupumzika vizuri. 6) Kupata Msaada Wa Kisaikolojia. Mimba inaweza kuleta mabadiliko mengi ya kihisia na kisaikolojia. Ni muhimu kwa mama mjamzito kupata msaada wa kisaikolojia au kuzungumza na wapendwa wake ikiwa anahisimsongo wa mawazoau wasiwasi. 7) Elimu Na Ushauri. Jifunze kuhusu ujauzito, kujifungua, na malezi ya mtoto. Pata ushauri wa kutosha kutoka kwa wataalamu wa afya na wale walio na uzoefu. HITIMISHO: Ni muhimu kwa mama mjamzito kufanya juhudi za kuzingatia afya yake yeye na mtoto wake aliye tumboni kwa pamoja. Kwa hiyo, kufuata miongozo ya afya ya uzazi na kushauriana na wataalamu', 'metadata': {'text': 'mpango wa mazoezi. 5) Kupata Usingizi Wa Kutosha. Usingizi wa kutosha ni muhimu kwa afya ya mama na mtoto aliye tumboni. Mama mjamzito anapaswa kupata usingizi wa kutosha na kupumzika vizur

In [27]:
# #Generate test set
# from giskard.rag import generate_testset

# testset = generate_testset(
#     knowledge_base,
#     num_questions=60,
#     agent_description="Chatbot ya Kiswahili anayetoa majibu kuhusu afya ya uzazi, hasa lishe bora kwa wanawake wajawazito"
# )

In [28]:
from giskard.rag import generate_testset

testset2 = generate_testset(
    knowledge_base,
    num_questions=60,
    language='sw',  # Force Swahili
    agent_description="Chatbot ya Kiswahili anayetoa majibu kuhusu afya ya uzazi, hasa lishe bora kwa wanawake wajawazito"
)


2025-05-24 14:30:37,793 pid:14944 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


c:\Users\DELL\anaconda3\envs\llmapp\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


2025-05-24 14:31:24,325 pid:14944 MainThread giskard.rag  INFO     Found 5 topics in the knowledge base.


Generating questions: 100%|██████████| 60/60 [03:22<00:00,  3.38s/it]


In [29]:
# Save the generated testset
testset2.save("my_testset2.jsonl")

In [30]:
test_set_df = testset2.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: Kwa nini maji ni muhimu kwa mwili?
Reference answer: Maji ni muhimu kwa mwili kwa sababu inasaidia kwenye mmenyenyo na unyonywaji chakula na pia husaidia kuzia choo.
Reference context:
Document 15: fatty acids ambayo inasaidia ukuaji wa ubongo na macho ya mtoto. Maji ni muhimu kwa mwili kwa sababu inasaidia kwenye mmenyenyo na unyonywaji chakula na pia husaidia kuzia choo
******************

Question 2: Ni aina gani ya vyakula vilivyotajwa katika muktadha huu?
Reference answer: Vilivyosindikwa (Processed Junk Foods).
Reference context:
Document 62: Vilivyosindikwa (Processed Junk Foods).
******************

Question 3: Kwa nini ni muhimu kwa mama mjamzito kupata usingizi wa kutosha?
Reference answer: Usingizi wa kutosha ni muhimu kwa afya ya mama na mtoto aliye tumboni.
Reference context:
Document 49: mpango wa mazoezi. 5) Kupata Usingizi Wa Kutosha. Usingizi wa kutosha ni muhimu kwa afya ya mama na mtoto aliye tumboni. Mama mjamzito anapaswa kupata usingizi wa kutosha na k

In [31]:
from langchain.prompts import PromptTemplate

template = """
Tumia maelezo katika muktadha hapa chini kujibu swali lililoulizwa. 
Ikiwa huwezi kupata jibu kutoka kwa muktadha, sema "Sijui". 
Usijaribu kubuni au kuongeza taarifa zisizotolewa.

Muktadha: {context}

Swali: {question}

Jibu lazima liwe kwa Kiswahili fasaha, na lizingatie mada ya afya ya uzazi au lishe ya wanawake wajawazito.
"""

prompt = PromptTemplate.from_template(template)

# Example usage
print(prompt.format(
    context="Wanawake wajawazito wanahitaji madini ya chuma ili kuepuka upungufu wa damu.",
    question="Kwa nini wanawake wajawazito wanahitaji madini ya chuma?"
))


Tumia maelezo katika muktadha hapa chini kujibu swali lililoulizwa. 
Ikiwa huwezi kupata jibu kutoka kwa muktadha, sema "Sijui". 
Usijaribu kubuni au kuongeza taarifa zisizotolewa.

Muktadha: Wanawake wajawazito wanahitaji madini ya chuma ili kuepuka upungufu wa damu.

Swali: Kwa nini wanawake wajawazito wanahitaji madini ya chuma?

Jibu lazima liwe kwa Kiswahili fasaha, na lizingatie mada ya afya ya uzazi au lishe ya wanawake wajawazito.



In [32]:
# Create a Retriver
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("Ni vyakula gani hatari kwa mjamzito?")

C:\Users\DELL\AppData\Local\Temp\ipykernel_14944\1757952414.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Ni vyakula gani hatari kwa mjamzito?")


[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250404204209', 'source': 'data\\Vyakula_Hatari_Kwa_Mama_Mjamzito.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Vyakula Hatari Kwa Mama Mjamzito. 1) Matunda Na Mboga Zisizooshwa Vizuri. Matunda na mboga zisizooshwa vizuri zinaweza kubeba mabaki ya dawa za kuulia wadudu na vimelea kama toxoplasma. Safisha vizuri matunda na mboga kabla ya kula. 2) Chakula Kibichi Au Kisichoiva Vizuri. Nyama mbichi au iliyopikwa lakini haikuiva vizuri, samaki mbichi, na mayai mabichi au vyakula vilivyochanganywa na mayai mabichi vinaweza kubeba bakteria na vimelea kama vile salmonella, listeria, na E. coli ambayo ni hatari kwa mama na mtoto. Mfano:Kuna baadhi ya Ice cream na mayonaizi hutengenezwa kwa kuchanganywa na mayai mabichi. Mjamzito amekatazwa kuvila vyakula hivyo. Mama mjamzito, Kabla ya kula hakikisha nyama, samaki, na mayai vinaiva vizuri. 3) Pombe. Hakuna

In [33]:
# Create chain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [34]:
chain.invoke({"question": "Ni nini umuhimu wa lishe bora kwa mjamzito?"})

'Umuhimu wa lishe bora kwa mjamzito ni kuhakikisha kuwa mama anapata virutubishi muhimu vinavyomsaidia kuimarisha afya yake na kuendelea vizuri kiafya pamoja na mtoto anayejifungua. Lishe bora huwezesha mama kupata uzito wa kutosha na kuzuia matatizo kama upungufu wa damu, mimba kuharibika, mtoto kufia tumboni, au uzito pungufu wa mtoto. Vilevile, lishe nzuri humsaidia mama kuwa na mfumo wa kinga imara, hivyo kupungua kwa hatari za magonjwa na matatizo yanayohusiana na ujauzito. Kila siku, mama mjamzito anashauriwa kula matunda, mboga, vyakula vya wanyama vyenye kalsiamu, na nafaka, ili kuzingatia mahitaji yake na kuhakikisha maendeleo mazuri ya kiafya kwa yeye na mtoto wake.'

### Evaluating the Model on the Test Set


In [35]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

In [36]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset2, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 60/60 [01:14<00:00,  1.24s/it]


Here are the five components of our RAG application:

- **Generator** : This is the LLM used in the chain to generate the answers.
- **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
- **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
- **Router**: This is a component that filters the query of the user based on his intentions.
- **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.


In [37]:
display(report)

Loading BokehJS ...

In [38]:
report.to_html("report2.html")

In [39]:
report.correctness_by_question_type()

,correctness
question_type,
complex,0.8
conversational,0.2
distracting element,0.4
double,0.9
simple,0.8
situational,0.9


In [40]:
failure = report.get_failures()

In [41]:
failure_df = pd.DataFrame(failure)
failure_df[0:1]

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
f7d55dd3-e145-40ff-85b2-4056fdb10686,Ni aina gani ya vyakula vilivyotajwa katika mu...,Vilivyosindikwa (Processed Junk Foods).,Document 62: Vilivyosindikwa (Processed Junk F...,[],"{'question_type': 'simple', 'seed_document_id'...",Aina za vyakula vilivyotajwa katika muktadha h...,False,The agent provided a detailed list of healthy ...
